In [11]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'Chapter3/housing.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
housing = pd.read_csv(file_content_stream)

## Defining inputs and target variable
X = housing.drop(['MEDV'], axis = 1)
Y = housing['MEDV']

## Splitting data into training, validation and testing
X_train, X_val, Y_train, Y_val =  train_test_split(X, Y, test_size = 0.3, random_state = 618, shuffle = True)
X_val, X_test, Y_val, Y_test = train_test_split(X_val, Y_val, test_size = 0.5, random_state = 620, shuffle = True)

In [19]:
import sagemaker
from sagemaker import image_uris

container = image_uris.retrieve(region = boto3.Session().region_name, framework = "linear-learner")

sess = sagemaker.Session()

linear = sagemaker.estimator.Estimator(container,
                                       role,
                                       instance_count = 1,
                                       instance_type = "ml.c4.xlarge",
                                       output_path = output_location,
                                       sagemaker_session = sess,
                                       )

linear.set_hyperparameters(feature_dim = 784, 
                           predictor_type = "binary_classifier", 
                           mini_batch_size = 200)

linear.fit({"train": s3_train_data})